In [ ]:
import pandas as pd
import requests
import datetime as dt
from dateutil import parser
import pytz
import json

In [ ]:
OPENFX_URL = "https://marginalttdemowebapi.fxopen.net:8443/api/v2"

API_ID = "xx"
API_KEY = "xx"
API_SECRET = "xx"

SECURE_HEADER = {
    "Authorization": f"Basic {API_ID}:{API_KEY}:{API_SECRET}",
    "Content-Type": "application/json",
    "Accept": "application/json",
}

In [ ]:
session = requests.Session()
session.headers.update(SECURE_HEADER)

In [ ]:
full_url = lambda x: f"{OPENFX_URL}/{x}"

## Account Details

In [ ]:
resp = session.get(full_url('account'))

In [ ]:
resp.status_code

In [ ]:
#print(json.dumps(resp.json(), indent=2))

## Symbols (Instruments)

In [ ]:
# first get all of the symbols
resp = session.get(full_url('symbol'))
symbol_data = resp.json()

# print first 2, note the StatusGroupId=="Forex"
[print(json.dumps(x, indent=2)) for x in symbol_data[:2]]

# also we only want symbols where we can also load history data. For that there is the quotehistory/symbols
resp = session.get(full_url('quotehistory/symbols'))
his_symbol_data = resp.json()

print(his_symbol_data[:5])

# you can probaby see, some of the instruments are appended with "_L"
# in the API code we will filter for symbols that are in the symbol_data and are in his_symbol_data and do not have this L and have StatusGroupId=="Forex"

## Perdiodicities (Granularities)

In [ ]:
# for a given instrument (symbol) we can get the available candle granularities
resp = session.get(full_url('quotehistory/EURUSD/periodicities'))
print(resp.json())

## Candles

Candles are a little bit different than Oanda. <br><br>
The Good:
- Prices are floats!!
- Last available candle is in the response<br><br>

The bad:
- We have to make separate calls for ask and bid prices
- We have to specify a from date no matter what. It has to be a timestamp in ms format without timezone.
    - If the count we specify is negative it counts back from the date
    - If the count we specify is postive it counts forward from the date
- There is a 1000 candle limit per request

In [ ]:
test_date = dt.datetime.utcnow()
past_date = parser.parse("2023-03-02T03:11:00")
print("test_date", test_date)
print("past_date", past_date)

test_date_ts = pd.Timestamp(test_date).timestamp()
past_date_ts = pd.Timestamp(past_date).timestamp()
print("test_date_ts", test_date_ts)
print("int(test_date_ts*1000)", int(test_date_ts*1000))
print("int(past_date_ts*1000)", int(past_date_ts*1000))

In [ ]:
ts_conv = 1640995200000
pd.to_datetime(ts_conv, unit='ms')

In [ ]:
LABEL_MAP = {
    'Open': 'o',
    'High': 'h',
    'Low': 'l',
    'Close': 'c',
}

# normal params
count = -10
granularity = "M15"
pair = "EURUSD"

# how far do we need to go back to get our candles
params = dict(
    timestamp=int(pd.Timestamp(dt.datetime.utcnow()).timestamp() * 1000),
    count=count
)

url = full_url(f'quotehistory/{pair}/{granularity}/bars/bid')
bid_data = session.get(url, params=params).json()

url = full_url(f'quotehistory/{pair}/{granularity}/bars/ask')
ask_data = session.get(url, params=params).json()


In [ ]:
# bid_data and ask_data will contain (hopefully) a key called "Bars", with the candle data:
bid_data["Bars"][:2] # the first two

In [ ]:
# Now to convert them to a dataframe
# main points: Timestamp to a datetime.
# we'll need to make a DataFrame for ask, for bid, merge and calculate the mid

In [ ]:
# a little utility to take in a candle and return it as and object
# for example, if we are working with bid prices
# price_label='bid'
# item= {'Volume': 1476, 'Close': 1.06064,  'Low': 1.06054,  'High': 1.06104,  'Open': 1.06081,  'Timestamp': 1677535200000}
# the returned object is: { 'time': datetime, 'bid_c': 1.06064,  'bid_l': 1.06054,  'bid_h': 1.06104,  'bid_o': 1.06081 }
def get_price_dict(price_label: str, item):
        data = dict(time=pd.to_datetime(item['Timestamp'], unit='ms'))
        for ohlc in LABEL_MAP.keys():
            data[f"{price_label}_{LABEL_MAP[ohlc]}"]=item[ohlc]
        return data

In [ ]:
# let's make the lists of objects
AvailableTo = pd.to_datetime(bid_data['AvailableTo'], unit='ms')

bids = [get_price_dict('bid', item) for item in bid_data["Bars"]]
asks = [get_price_dict('ask', item) for item in ask_data["Bars"]]

In [ ]:
# last 2
bids[-2:]

In [ ]:
# now merge on time - the assumption here is we have the same time values for both. it would be weird if we didn't
df_bid = pd.DataFrame.from_dict(bids)
df_ask = pd.DataFrame.from_dict(asks)
df_merged = pd.merge(left=df_bid, right=df_ask, on='time')    

In [ ]:
df_merged

In [ ]:
# FINALLY calcuate the mid, and we are done
for i in ['_o', '_h', '_l', '_c']:
    df_merged[f'mid{i}'] = (df_merged[f'ask{i}'] - df_merged[f'bid{i}']) / 2 + df_merged[f'bid{i}']

In [ ]:
df_merged

In [ ]:
if count < 0 and df_merged.shape[0] > 0 and df_merged.iloc[-1].time == AvailableTo:
    df_merged = df_merged[:-1]

In [ ]:
df_merged

In [ ]:
# and breathe...

## Latest Prices

In [ ]:
# here the endpoint needs space separated instruments rather than comma. Yes, in a URL that is a bit weird.
instruments_list = ["GBPJPY", "EURUSD", "EURNOK"]
url = full_url(f"tick/{' '.join(instruments_list)}")
print(url)

In [ ]:
prices = session.get(url)
price_data = prices.json()

# you can see below that there are some differences to the Oanda Api in what comes back, imho this is a lot better
price_data